In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

input_ = pd.read_excel('./part_number.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[['JOIN_MPN', 'Manufacturer', 'Part_Number']]
input_.drop_duplicates(ignore_index=True)

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

import numpy as np

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                url = 'https://www.rockauto.com/en/partsearch/?mfr=' + input_.loc[i, 'Manufacturer'] + '&partnum=' + input_.loc[i, 'Part_Number']

                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))
                
                security_code = html.xpath('//div[@class="center"]/div')
                
                # = = = = = = = = = = = = = = =
                
                if len(security_code) != 0 and 'security code' in security_code[0].text:
                    continue
                
                # = = = = = = = = = = = = = = =

                list_part = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

                # = = = = = = = = = = = = = = =
                
                list_manufacturer = [part.xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "listing-final-manufacturer")]')[0].text.strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                manufacturer_index = np.where(np.array(list_manufacturer) == input_.loc[i, 'Manufacturer'])[0]

                # = = = = = = = = = = = = = = =
                
                list_part_number = [part.xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@title, "Buyer\'s Guide")]')[0].text.strip() for part in list_part]
                
                # = = = = = = = = = = = = = = =
                
                part_number_index = np.where(np.array(list_part_number) == input_.loc[i, 'Part_Number'])[0]
                
                # = = = = = = = = = = = = = = =
                
                part_index = list(np.intersect1d(manufacturer_index, part_number_index))
                
                # = = = = = = = = = = = = = = =
                
                if len(part_index) == 1:
                    part_index = part_index[0]
                    
                    # = = = = = = = = = = = = = = =
                
                    list_note_1 = list_part[part_index].xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "span-link-underline-remover")]/text()')

                    note_1 = '' if len(list_note_1) == 0 else ';'.join([note_1.strip() for note_1 in list_note_1])

                    # = = = = = = = = = = = = = = =
                
                    list_note_2 = list_part[part_index].xpath('./descendant::span[contains(@title, "Please make sure")]/span/text()')

                    note_2 = '' if len(list_note_2) == 0 else ';'.join([note_2.strip() for note_2 in list_note_2])

                    # = = = = = = = = = = = = = = =
                    
                    src = list_part[part_index].xpath('./descendant::img[contains(@id, "inlineimg_thumb")]')

                    src = '' if len(src) == 0 else 'https://www.rockauto.com' + src[0].xpath('./@src')[0].strip().replace('__ra_m.', '.')

                    # = = = = = = = = = = = = = = =
                    
                    oe = list_part[part_index].xpath('./descendant::span[contains(@title, "Replaces these Alternate")]')

                    oe = '' if len(oe) == 0 else oe[0].xpath('./text()')[0].strip().replace(', ', ';')

                    # = = = = = = = = = = = = = = =
                    
                    part_json = json.loads(list_part[part_index].xpath('./descendant::input[contains(@id, "listing_data_essential")]/@value')[0])

                    part_type = part_json['parttype'].strip()
                    part_key = part_json['partkey'].strip()

                    # = = = = = = = = = = = = = = =
                    
                    df_temp = pd.DataFrame([{'JOIN_MPN': input_.loc[i, 'JOIN_MPN'],
                                             'OE': oe,
                                             'Note_1': note_1,
                                             'Note_2': note_2,
                                             'Pic': '',
                                             'Url': url,
                                             'Info': 'https://www.rockauto.com/en/moreinfo.php?pt=' + part_type + '&pk=' + part_key,
                                             'Src': src,
                                             'Part_Type': part_type,
                                             'Part_Key': part_key}])

                    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                    # = = = = = = = = = = = = = = =
                
                    crawler_status = 'ok'

                    break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status != 'ok':
            df_temp = pd.DataFrame([{'JOIN_MPN': input_.loc[i, 'JOIN_MPN'],
                                     'Manufacturer': input_.loc[i, 'Manufacturer'],
                                     'Part_Number': input_.loc[i, 'Part_Number']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Manufacturer'] + ' - ' + input_.loc[i, 'Part_Number'] + '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['JOIN_MPN'], ascending=[True])
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['JOIN_MPN'], ascending=[True])
    output_error.to_excel('./part_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：475

MELLING - 30719SHWP  <->  [ok] - 剩余数量：455 - 当前时间：13:36:07
MELLING - 30490S  <->  [ok] - 剩余数量：454 - 当前时间：13:36:07
MELLING - 30490SA  <->  [ok] - 剩余数量：453 - 当前时间：13:36:07
MELLING - 31035S  <->  [ok] - 剩余数量：452 - 当前时间：13:36:07
MELLING - 30460SA  <->  [ok] - 剩余数量：451 - 当前时间：13:36:07
MELLING - 30430S  <->  [ok] - 剩余数量：450 - 当前时间：13:36:07
MELLING - 30490SX  <->  [ok] - 剩余数量：449 - 当前时间：13:36:08
MELLING - 30460SB  <->  [ok] - 剩余数量：448 - 当前时间：13:36:08
MELLING - 30250S  <->  [ok] - 剩余数量：447 - 当前时间：13:36:08
MELLING - 30250SX  <->  [ok] - 剩余数量：446 - 当前时间：13:36:08
MELLING - 31036SA  <->  [ok] - 剩余数量：445 - 当前时间：13:36:08
MELLING - 31036S  <->  [ok] - 剩余数量：444 - 当前时间：13:36:08
MELLING - 31038SX  <->  [ok] - 剩余数量：443 - 当前时间：13:36:09
MELLING - 31041S  <->  [ok] - 剩余数量：442 - 当前时间：13:36:09
MELLING - 31045SA  <->  [ok] - 剩余数量：441 - 当前时间：13:36:09
MELLING - 31047SH  <->  [ok] - 剩余数量：440 - 当前时间：13:36:09
MELLING - 31045S  <->  [ok] - 剩余数量：439 - 当前时间：13:36:09
MELLING - 30470S  <->  [ok] - 剩余数量：438 - 当前时

/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_2813/1723183838.py:87: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  manufacturer_index = np.where(np.array(list_manufacturer) == input_.loc[i, 'Manufacturer'])[0]
/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_2813/1723183838.py:95: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  part_number_index = np.where(np.array(list_part_number) == input_.loc[i, 'Part_Number'])[0]


MELLING - 30470SX  <->  [ok] - 剩余数量：417 - 当前时间：13:36:18
MELLING - 31049SDX  <->  [ok] - 剩余数量：416 - 当前时间：13:36:18
MELLING - 31049SH  <->  [ok] - 剩余数量：415 - 当前时间：13:36:19
MELLING - 31049SCXH  <->  [ok] - 剩余数量：414 - 当前时间：13:36:19
MELLING - 31049SG  <->  [ok] - 剩余数量：413 - 当前时间：13:36:19
MELLING - 30490SAX  <->  [ok] - 剩余数量：412 - 当前时间：13:36:19
MELLING - 31049SJ  <->  [ok] - 剩余数量：411 - 当前时间：13:36:19
MELLING - 31049SC  <->  [ok] - 剩余数量：410 - 当前时间：13:36:19
MELLING - 31049SL  <->  [ok] - 剩余数量：409 - 当前时间：13:36:20
MELLING - 31049SA  <->  [ok] - 剩余数量：408 - 当前时间：13:36:20
MELLING - 31049SN  <->  [ok] - 剩余数量：407 - 当前时间：13:36:21
MELLING - 31051S  <->  [ok] - 剩余数量：406 - 当前时间：13:36:21
MELLING - 31049SVVT1  <->  [ok] - 剩余数量：405 - 当前时间：13:36:21
MELLING - 31051SA  <->  [ok] - 剩余数量：404 - 当前时间：13:36:22
MELLING - 31049SW  <->  [ok] - 剩余数量：403 - 当前时间：13:36:22
MELLING - 31049SPVVT  <->  [ok] - 剩余数量：402 - 当前时间：13:36:23
MELLING - 31054S  <->  [ok] - 剩余数量：401 - 当前时间：13:36:23
MELLING - 31051SAX  <->  [ok] - 剩余数量：400

MELLING - 3391SCCVVT  <->  [ok] - 剩余数量：269 - 当前时间：13:37:13
MELLING - 3391SA  <->  [ok] - 剩余数量：268 - 当前时间：13:37:14
MELLING - 3391S  <->  [ok] - 剩余数量：267 - 当前时间：13:37:14
MELLING - 3391SCVVTH  <->  [ok] - 剩余数量：266 - 当前时间：13:37:15
MELLING - 3376SA  <->  [ok] - 剩余数量：265 - 当前时间：13:37:16
MELLING - 3391SCCK3A  <->  [ok] - 剩余数量：264 - 当前时间：13:37:16
MELLING - 3391SCWP  <->  [ok] - 剩余数量：263 - 当前时间：13:37:17
MELLING - 3391SCCWP  <->  [ok] - 剩余数量：262 - 当前时间：13:37:17
MELLING - 3391SCWP1  <->  [ok] - 剩余数量：261 - 当前时间：13:37:17
MELLING - 3391SCVVT  <->  [ok] - 剩余数量：260 - 当前时间：13:37:17
MELLING - 3391SE  <->  [ok] - 剩余数量：259 - 当前时间：13:37:18
MELLING - 3391SD  <->  [ok] - 剩余数量：258 - 当前时间：13:37:18
MELLING - 3391SFH  <->  [ok] - 剩余数量：257 - 当前时间：13:37:18
MELLING - 3391SCK3  <->  [ok] - 剩余数量：256 - 当前时间：13:37:19
MELLING - 3392S  <->  [ok] - 剩余数量：255 - 当前时间：13:37:19
MELLING - 3391SXH  <->  [ok] - 剩余数量：254 - 当前时间：13:37:20
MELLING - 3393SA  <->  [ok] - 剩余数量：253 - 当前时间：13:37:20
MELLING - 3393SAH  <->  [ok] - 剩余数量：252 

MELLING - 3705SB  <->  [ok] - 剩余数量：122 - 当前时间：13:38:11
MELLING - 3707SAX  <->  [ok] - 剩余数量：121 - 当前时间：13:38:11
MELLING - 3705SBX  <->  [ok] - 剩余数量：120 - 当前时间：13:38:11
MELLING - 3391SCCWP1  <->  [ok] - 剩余数量：119 - 当前时间：13:38:12
MELLING - 3503S  <->  [ok] - 剩余数量：118 - 当前时间：13:38:12
MELLING - 3708SA  <->  [ok] - 剩余数量：117 - 当前时间：13:38:13
MELLING - 3707SX  <->  [ok] - 剩余数量：116 - 当前时间：13:38:14
MELLING - 3711SX  <->  [ok] - 剩余数量：115 - 当前时间：13:38:14
MELLING - 3715SAX  <->  [ok] - 剩余数量：114 - 当前时间：13:38:14
MELLING - 3715SA  <->  [ok] - 剩余数量：113 - 当前时间：13:38:16
MELLING - 3715S  <->  [ok] - 剩余数量：112 - 当前时间：13:38:16
MELLING - 3720S  <->  [ok] - 剩余数量：111 - 当前时间：13:38:16
MELLING - 3711SBX  <->  [ok] - 剩余数量：110 - 当前时间：13:38:17
MELLING - 3715SX  <->  [ok] - 剩余数量：109 - 当前时间：13:38:17
MELLING - 3708S  <->  [ok] - 剩余数量：108 - 当前时间：13:38:17
MELLING - 3720SA  <->  [ok] - 剩余数量：107 - 当前时间：13:38:18
MELLING - 3720SAXHWP  <->  [ok] - 剩余数量：106 - 当前时间：13:38:19
MELLING - 3720SAX  <->  [ok] - 剩余数量：105 - 当前时间：13:38:19
M